In [1]:
import pandas as pd
import numpy as np

In [2]:
!pip install cvxpy

     |████████████████████████████████| 952kB 3.3MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 163kB 7.2MB/s eta 0:00:01
     |████████████████████████████████| 215kB 15.4MB/s eta 0:00:01
     |████████████████████████████████| 153kB 11.3MB/s eta 0:00:01
     |████████████████████████████████| 1.6MB 18.0MB/s eta 0:00:01
  Created wheel for cvxpy: filename=cvxpy-1.0.28-cp36-cp36m-linux_x86_64.whl size=2093852 sha256=cdf86b97c4aafdcd4dafb2c8ef19effbb2ee31b110780eae7c47f229307a9e4f
  Stored in directory: /home/nbuser/.cache/pip/wheels/c0/39/27/b05cd1ca18445ed4b54c8fe7ecae57dae188cc55b7dd2bbfbf
Successfully built cvxpy
  Created wheel for scs: filename=scs-2.1.1.post2-cp36-cp36m-linux_x86_64.whl size=371830 sha256=28bd33c392a0b99d23170e8c5b3df6cc7b8dcd1b8ba3e88de72b77ae587789a6
  Stored in directory: /home/nbuser/.cache/pip/wheels/68/3f/24/e9c75d426f60063

In [3]:
import cvxpy as cp

# Read in finish probabilities

### Option 1: scrape from website

In [4]:
# import requests

# page = requests.get('https://datagolf.ca/pga-tour-predictions')
# page.status_code

# info = str(page.content) 
# #use str(page.content), not page.text.  Otherwise ast.literal_eval doesn't work
# #info[:100]

# #Can we just find the dict we care about?
# #data we want looks like:
# # var probs = JSON.parse('[{....}]')
# import re
# d = re.findall("var probs = JSON\.parse(.*?)]",info)
# #type(d)

# import ast
# e = ast.literal_eval(str(d[0])[5:])
# #type(e)

In [5]:
# #Now build out df
# df2 = pd.DataFrame(e[0],index=np.arange(1))
# df2.head()

In [6]:
# for i in range(1,len(e)):
#     p = e[i]
#     df2 = pd.concat([df2,pd.DataFrame(e[i],index=np.arange(1))])
# df2.head()

In [7]:
# #delete columns we don't care about
# df2.drop(['event_name','flag','sample_size'],axis=1,inplace=True)
# df2.head(2)

In [8]:
# df2.head(20)

### Option 2: Import 

In [9]:
euro = False
alt = False #pga alternate field

In [10]:
if euro:
    df2=pd.read_csv('Data/Euro_preds.csv')
elif alt:
    df2= pd.read_csv('Data/puerto_rico_open_preds.csv')
else:
    df2 = pd.read_csv('Data/pga_probs_20200308_Players.csv')
df2.head()

,player_name,sample_size,win,top_5,top_10,top_20,make_cut
0,Rory McIlroy,150,0.080675,0.252950,0.384225,0.549725,0.82690
1,Jon Rahm,150,0.067150,0.225900,0.352275,0.518550,0.81595
2,Justin Thomas,150,0.044550,0.166750,0.275450,0.433325,0.76655
3,Hideki Matsuyama,150,0.033400,0.132975,0.228950,0.378175,0.73925
4,Xander Schauffele,150,0.026125,0.111525,0.196650,0.334675,0.70960


In [11]:
#Add field with name of First Last
if euro or alt:
    df2['temp_Name'] = df2.player_name
    temp_name = df2.player_name.str.split(', ',expand=True,n=2)
    df2['temp_Name'] = temp_name[1] + " " + temp_name[0]
else:
    df2['temp_Name'] = df2.player_name
#     temp_name = df2.player_name.str.split(', ',expand=True,n=2)
#     df2['temp_Name'] = temp_name[1] + " " + temp_name[0]
df2.head()

,player_name,sample_size,win,top_5,top_10,top_20,make_cut,temp_Name
0,Rory McIlroy,150,0.080675,0.252950,0.384225,0.549725,0.82690,Rory McIlroy
1,Jon Rahm,150,0.067150,0.225900,0.352275,0.518550,0.81595,Jon Rahm
2,Justin Thomas,150,0.044550,0.166750,0.275450,0.433325,0.76655,Justin Thomas
3,Hideki Matsuyama,150,0.033400,0.132975,0.228950,0.378175,0.73925,Hideki Matsuyama
4,Xander Schauffele,150,0.026125,0.111525,0.196650,0.334675,0.70960,Xander Schauffele


In [12]:
#Add sal
#df2_sal = pd.read_csv('Data/DKSalaries Memorial 2019.csv')
if euro:
    url = 'https://www.draftkings.com/lineup/getavailableplayerscsv?contestTypeId=29&draftGroupId=34298'
elif alt:
    url = 'https://www.draftkings.com/lineup/getavailableplayerscsv?contestTypeId=29&draftGroupId=33859'
else: 
    url = 'https://www.draftkings.com/lineup/getavailableplayerscsv?contestTypeId=29&draftGroupId=33970'
df2_sal = pd.read_csv(url)
df2_sal.head(2)

,Position,Name + ID,Name,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame
0,G,Rory McIlroy (14421198) (LOCKED),Rory McIlroy,14421198,G,11700,THE PLAYERS Championship,Golf,103.08
1,G,Jon Rahm (14421199) (LOCKED),Jon Rahm,14421199,G,11000,THE PLAYERS Championship,Golf,99.42


In [13]:
#To make sure names match, replace all names in df2 with actual DK names in df2_sal
from difflib import get_close_matches
df2['Name'] = ""
for i in np.arange(len(df2)):
    curr_name = df2['temp_Name'].iloc[i]
    try:
        new_name = get_close_matches(curr_name,df2_sal['Name'],cutoff=.8)[0]
        if new_name != curr_name:
            print("{} replaced with {}".format(curr_name,new_name))
    except:
        new_name = 'No Name'
        print('NO MATCH: curr_name = {}, new_name = {}'.format(curr_name,new_name))
    df2['Name'].iloc[i] = new_name
#df2['Name'] = df2['Name'].apply(lambda x: get_close_matches(x,df2_sal['Name']))
df2.head(5)

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,player_name,sample_size,win,top_5,top_10,top_20,make_cut,temp_Name,Name
0,Rory McIlroy,150,0.080675,0.252950,0.384225,0.549725,0.82690,Rory McIlroy,Rory McIlroy
1,Jon Rahm,150,0.067150,0.225900,0.352275,0.518550,0.81595,Jon Rahm,Jon Rahm
2,Justin Thomas,150,0.044550,0.166750,0.275450,0.433325,0.76655,Justin Thomas,Justin Thomas
3,Hideki Matsuyama,150,0.033400,0.132975,0.228950,0.378175,0.73925,Hideki Matsuyama,Hideki Matsuyama
4,Xander Schauffele,150,0.026125,0.111525,0.196650,0.334675,0.70960,Xander Schauffele,Xander Schauffele


In [14]:
#Find where DK Name ('Name') doesn't match the name from datagolf ('temp_Name')
#df2[df2.Name != df2.temp_Name]

In [15]:
#Add salary and  to df2
df2 = pd.merge(df2,df2_sal[['Name','Name + ID','Salary']],how='left',on='Name')
df2.dropna(inplace=True)
df2.head()

,player_name,sample_size,win,top_5,top_10,top_20,make_cut,temp_Name,Name,Name + ID,Salary
0,Rory McIlroy,150,0.080675,0.252950,0.384225,0.549725,0.82690,Rory McIlroy,Rory McIlroy,Rory McIlroy (14421198) (LOCKED),11700
1,Jon Rahm,150,0.067150,0.225900,0.352275,0.518550,0.81595,Jon Rahm,Jon Rahm,Jon Rahm (14421199) (LOCKED),11000
2,Justin Thomas,150,0.044550,0.166750,0.275450,0.433325,0.76655,Justin Thomas,Justin Thomas,Justin Thomas (14421200) (LOCKED),10800
3,Hideki Matsuyama,150,0.033400,0.132975,0.228950,0.378175,0.73925,Hideki Matsuyama,Hideki Matsuyama,Hideki Matsuyama (14421211) (LOCKED),8800
4,Xander Schauffele,150,0.026125,0.111525,0.196650,0.334675,0.70960,Xander Schauffele,Xander Schauffele,Xander Schauffele (14421206) (LOCKED),9400


In [16]:
#Copy
df_all = df2.copy()
df_all.sort_values(by='Salary',ascending=False).head(10)

,player_name,sample_size,win,top_5,top_10,top_20,make_cut,temp_Name,Name,Name + ID,Salary
0,Rory McIlroy,150,0.080675,0.252950,0.384225,0.549725,0.826900,Rory McIlroy,Rory McIlroy,Rory McIlroy (14421198) (LOCKED),11700
1,Jon Rahm,150,0.067150,0.225900,0.352275,0.518550,0.815950,Jon Rahm,Jon Rahm,Jon Rahm (14421199) (LOCKED),11000
2,Justin Thomas,150,0.044550,0.166750,0.275450,0.433325,0.766550,Justin Thomas,Justin Thomas,Justin Thomas (14421200) (LOCKED),10800
18,Brooks Koepka,150,0.008500,0.049100,0.100400,0.197650,0.594400,Brooks Koepka,Brooks Koepka,Brooks Koepka (14421202) (LOCKED),10200
12,Dustin Johnson,150,0.013200,0.066150,0.127400,0.240175,0.630375,Dustin Johnson,Dustin Johnson,Dustin Johnson (14421203) (LOCKED),10000
6,Patrick Cantlay,150,0.021175,0.099625,0.179775,0.314475,0.695475,Patrick Cantlay,Patrick Cantlay,Patrick Cantlay (14421204) (LOCKED),9800
9,Adam Scott,150,0.022825,0.100025,0.182750,0.315650,0.691300,Adam Scott,Adam Scott,Adam Scott (14421205) (LOCKED),9600
4,Xander Schauffele,150,0.026125,0.111525,0.196650,0.334675,0.709600,Xander Schauffele,Xander Schauffele,Xander Schauffele (14421206) (LOCKED),9400
8,Webb Simpson,150,0.027425,0.115400,0.205800,0.350125,0.718400,Webb Simpson,Webb Simpson,Webb Simpson (14421207) (LOCKED),9200
5,Bryson DeChambeau,150,0.025050,0.106175,0.189675,0.325600,0.698500,Bryson DeChambeau,Bryson DeChambeau,Bryson DeChambeau (14421208) (LOCKED),9100


# Optimize

In [17]:
#Set team parameters
lock = ['Patrick Cantlay']
#lock = ['Matt Wallace']
#lock = ['Rory Sabbatini','Danny Willett']
lock = ['Jon Rahm','Justin Thomas']
lock = []
lock_full_names = []
for i, p in enumerate(lock):
    lock_full_names.append(df_all[df_all.Name==p]['Name + ID'].iloc[0])
sal_used = df_all[df_all.Name.isin(lock)].Salary.sum()
sal_cap = 50000
sal_left = sal_cap-sal_used
players_left = 6-len(lock)


to_remove = ['Julian Suri','Adrian Otaegui','Matt Kuchar','Fabrizio Zanotti','Jordan L. Smith',
            'Richard Sterne','Tapio Pulkkanen','Matt Jones']
to_remove = ['Martin Kaymer','Shane Lowry','Francesco Molinari','Justin Harding','Alex Noren',
            'Tom Lewis','C.T. Pan','Lee Westwood','Edoardo Molinari','Hao-Tong Li']
to_remove = ['Ian Poulter', 'Jordan L. Smith','Richard Sterne','David Lipsky']
to_remove = ['Tyrrell Hatton','Paul Casey', 'Sergio Garcia', 'Shane Lowry']
to_remove = ['Abraham Ancer','Billy Horschel','Jim Furyk','Matt Kuchar',
            'Jason Day', 'Louis Oosthuizen', 'Sergio Garcia','Justin Rose',
             'Henrik Stenson','Rickie Fowler', 'Paul Casey']
#to_remove = []

min_sal = 6000
max_sal = 10000


In [18]:
#Opt by made cut (or T10 if no cut)
opt_teams = []
df_now = df_all.copy()


#Filter down remaining players by sal
df_now = df_now[(df_now.Salary>=min_sal) &
               (df_now.Salary<=max_sal)]
df_now = df_now[~df_now.Name.isin(lock)]

df_now = df_now[~ df_now.Name.isin(to_remove)]

num_teams = 10
for j in np.arange(num_teams):
    s = df_now.Salary.values
    c = df_now.make_cut*100
    T20 = df_now['top_20']*100
    T5 = df_now['top_5']*100
    T10 = df_now['top_10']
    w = df_now['win']*100
    n = len(df_now)
    #print(n,len(c),len(s))
    x = cp.Variable(n,boolean=True)
    cap = sal_cap
    constraints = [cp.sum(x) == players_left,
                 x*s <= sal_left]

    # Form objective.
    obj = cp.Maximize(x*c)
    #obj2 = cp.Maximize(x*T20)
    #obj3 = cp.Maximize(x*c)

    # Form and solve problem.
    #T10
    prob = cp.Problem(obj, constraints)
    prob.solve()  # Returns the optimal value.
    opt_team = df_now['Name'][x.value>.5].tolist()
    opt_teams.append(opt_team)
    #print('Team {}:'.format(j+1),opt_team)
    print('Team {} (obj = {:.1f}):'.format(j+1,prob.value),opt_team)

    
    #delet opt team
    #print(len(df_now))
    df_now = df_now[~df_now['Name'].isin(opt_team)]
    #print(len(df_now))

Team 1 (obj = 415.1): ['Hideki Matsuyama', 'Tommy Fleetwood', 'Webb Simpson', 'Collin Morikawa', 'Tyrrell Hatton', 'Matthew Fitzpatrick']
Team 2 (obj = 399.9): ['Xander Schauffele', 'Bryson DeChambeau', 'Sungjae Im', 'Patrick Reed', 'Scottie Scheffler', 'Joel Dahmen']
Team 3 (obj = 382.0): ['Patrick Cantlay', 'Adam Scott', 'Gary Woodland', 'Byeong Hun An', 'Daniel Berger', 'Ryan Moore']
Team 4 (obj = 353.8): ['Dustin Johnson', 'Tony Finau', 'Marc Leishman', 'Shane Lowry', 'Brandt Snedeker', 'Viktor Hovland']
Team 5 (obj = 333.1): ['Harris English', 'Nick Taylor', 'Corey Conners', 'Adam Hadwin', 'Vaughn Taylor', 'Chez Reavie']
Team 6 (obj = 326.5): ['Ian Poulter', 'Rory Sabbatini', 'Brian Harman', 'Kevin Kisner', 'Kevin Na', 'Brendon Todd']
Team 7 (obj = 314.8): ['Ryan Palmer', 'Harold Varner III', 'Christiaan Bezuidenhout', 'Bud Cauley', 'J.T. Poston', 'Tom Hoge']
Team 8 (obj = 308.3): ['Jason Kokrak', 'Max Homa', 'Erik Van Rooyen', 'Joaquin Niemann', 'Sebastian Munoz', 'Russell Knox']

In [19]:
#Opt by top 20
opt_teams_T20 = []
df_now = df_all.copy()

#Filter down remaining players by sal
df_now = df_now[(df_now.Salary>=min_sal) &
               (df_now.Salary<=max_sal)]
df_now = df_now[~df_now.Name.isin(lock)]

df_now = df_now[~ df_now.Name.isin(to_remove)]

#num_teams = 5
for j in np.arange(num_teams):
    s = df_now.Salary.values
    c = df_now.make_cut*100
    T20 = df_now['top_20']*100
    T10 = df_now['top_10']*100
    w = df_now['win']*100
    n = len(df_now)
    #print(n,len(c),len(s))
    x = cp.Variable(n,boolean=True)
    cap = 50000
    constraints = [cp.sum(x) == players_left,
                 x*s <= sal_left]

    # Form objective.
    obj = cp.Maximize(x*T20)
    #obj2 = cp.Maximize(x*T20)
    #obj3 = cp.Maximize(x*c)

    prob = cp.Problem(obj, constraints)
    prob.solve()  # Returns the optimal value.
    opt_team = df_now['Name'][x.value>.5].tolist()
    opt_teams_T20.append(opt_team)
    print('Team {} (obj = {:.1f}):'.format(j+1,prob.value),opt_team)
    
    #delet opt team
    #print(len(df_now))
    df_now = df_now[~df_now['Name'].isin(opt_team)]
    #print(len(df_now))

Team 1 (obj = 188.2): ['Hideki Matsuyama', 'Tommy Fleetwood', 'Webb Simpson', 'Collin Morikawa', 'Tyrrell Hatton', 'Matthew Fitzpatrick']
Team 2 (obj = 170.1): ['Xander Schauffele', 'Bryson DeChambeau', 'Sungjae Im', 'Patrick Reed', 'Scottie Scheffler', 'Joel Dahmen']
Team 3 (obj = 148.3): ['Patrick Cantlay', 'Adam Scott', 'Gary Woodland', 'Byeong Hun An', 'Daniel Berger', 'Ryan Moore']
Team 4 (obj = 116.2): ['Dustin Johnson', 'Tony Finau', 'Marc Leishman', 'Shane Lowry', 'Brandt Snedeker', 'Viktor Hovland']
Team 5 (obj = 96.1): ['Harris English', 'Nick Taylor', 'Adam Hadwin', 'Ian Poulter', 'Vaughn Taylor', 'Chez Reavie']
Team 6 (obj = 91.5): ['Corey Conners', 'Rory Sabbatini', 'Brian Harman', 'Kevin Kisner', 'Kevin Na', 'Brendon Todd']
Team 7 (obj = 82.4): ['Jason Kokrak', 'Ryan Palmer', 'Christiaan Bezuidenhout', 'Bud Cauley', 'J.T. Poston', 'Tom Hoge']
Team 8 (obj = 77.1): ['Max Homa', 'Erik Van Rooyen', 'Harold Varner III', 'Carlos Ortiz', 'Sebastian Munoz', 'Russell Knox']
Team 9

In [20]:
for n in np.arange(1,num_teams+1):
    print('In top {} teams:'.format(n))
    cut_not_t20 = set(np.hstack(opt_teams[:n])) - set(np.hstack(opt_teams_T20[:n]))
    t20_not_cut = set(np.hstack(opt_teams_T20[:n])) - set(np.hstack(opt_teams[:n]))
    print('\tIn made-cut, not Top 20:',cut_not_t20)
    print('\tIn Top 20, not made-cut:',t20_not_cut)

In top 1 teams:
	In made-cut, not Top 20: set()
	In Top 20, not made-cut: set()
In top 2 teams:
	In made-cut, not Top 20: set()
	In Top 20, not made-cut: set()
In top 3 teams:
	In made-cut, not Top 20: set()
	In Top 20, not made-cut: set()
In top 4 teams:
	In made-cut, not Top 20: set()
	In Top 20, not made-cut: set()
In top 5 teams:
	In made-cut, not Top 20: {'Corey Conners'}
	In Top 20, not made-cut: {'Ian Poulter'}
In top 6 teams:
	In made-cut, not Top 20: set()
	In Top 20, not made-cut: set()
In top 7 teams:
	In made-cut, not Top 20: {'Harold Varner III'}
	In Top 20, not made-cut: {'Jason Kokrak'}
In top 8 teams:
	In made-cut, not Top 20: {'Joaquin Niemann'}
	In Top 20, not made-cut: {'Carlos Ortiz'}
In top 9 teams:
	In made-cut, not Top 20: set()
	In Top 20, not made-cut: set()
In top 10 teams:
	In made-cut, not Top 20: {'Danny Lee'}
	In Top 20, not made-cut: {'Talor Gooch'}


In [21]:
# num_teams = 3
# cut_not_t20= set(np.hstack(opt_teams[:num_teams])) - set(np.hstack(opt_teams_T20[:num_teams]))
# print('In made-cut team, not Top 20:',cut_not_t20)

In [22]:
# print('In top 20, not made-cut:')
# print(set(np.hstack(opt_teams_T20[:num_teams])) - set(np.hstack(opt_teams[:num_teams])))

## Creat lineups as csv

In [23]:
#Assign exposures for top j teams
teams_to_keep = 4
own_gap = 6
exposures = list(np.arange(50,50-own_gap*teams_to_keep,-own_gap))
if len(exposures)<len(opt_teams_T20):
    exposures = exposures + [0]*(len(opt_teams_T20)-len(exposures))
exposures

[50, 44, 38, 32, 0, 0, 0, 0, 0, 0]

In [24]:
#Create df to hold player pool
df_teams = df2[['Name + ID','Name','Salary']]
df_teams.sort_values(by='Salary',ascending=False,inplace=True)
df_teams['Exposure'] = 0
df_teams.head(2)

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Name + ID,Name,Salary,Exposure
0,Rory McIlroy (14421198),Rory McIlroy,11700,0
1,Jon Rahm (14421199),Jon Rahm,11000,0


In [25]:
#Use opt_teams or opt_teams_T20
for i, team in enumerate(opt_teams):
    df_teams.Exposure = np.where(df_teams.Name.isin(team),exposures[i],df_teams.Exposure)

#Add more players
#add those in made cut
num_teams = len([e for e in exposures if e>0])
cut_not_t20 = set(np.hstack(opt_teams[:num_teams])) - set(np.hstack(opt_teams_T20[:num_teams]))
#more_players= {'Robert Macintyre':49}
more_players={p:39 for p in cut_not_t20}
#more_players = {}
# more_players['Victor Perez']=40
# more_players['Adri Arnaus']=40
# more_players['Andrea Pavan']=40
more_players = {}
for p in list(more_players.keys()):
    df_teams.Exposure = np.where(df_teams.Name==p,more_players[p],df_teams.Exposure)
    
df_teams = df_teams[df_teams.Exposure > 0]
df_teams['Actual Exposure'] = 0
df_teams.reset_index(drop=True,inplace=True)
df_teams

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/pandas/core/generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


,Name + ID,Name,Salary,Exposure,Actual Exposure
0,Dustin Johnson (14421203),Dustin Johnson,10000,32,0
1,Patrick Cantlay (14421204),Patrick Cantlay,9800,38,0
2,Adam Scott (14421205),Adam Scott,9600,38,0
3,Xander Schauffele (14421206),Xander Schauffele,9400,44,0
4,Webb Simpson (14421207),Webb Simpson,9200,50,0
5,Bryson DeChambeau (14421208),Bryson DeChambeau,9100,44,0
6,Tommy Fleetwood (14421209),Tommy Fleetwood,9000,50,0
7,Hideki Matsuyama (14421211),Hideki Matsuyama,8800,50,0
8,Sungjae Im (14421213),Sungjae Im,8600,44,0
9,Patrick Reed (14421214),Patrick Reed,8500,44,0


In [26]:
more_players

{}

In [27]:
#initialize df_lineups, which will be written to csv
df_lineups = pd.DataFrame(columns=['G1','G2','G3','G4','G5','G6'])
df_lineups

,G1,G2,G3,G4,G5,G6


In [28]:
#First, need a function that picks elements from list L with probabilies probs
def pick_element_per_probs(L,probs):
    #make sure probs are normalized
    probs = [x / sum(probs) for x in probs]
    #pick a random number
    r = np.random.random()
    c = 0 #cumulative sum
    for i, prob in enumerate(probs):
        if r < prob + c:
            return L[i]
        c += prob

In [29]:
#Now start churning
#Now set parameters for number of teams and salary limits
num_teams = 45
min_sal = 49400 - sal_used
max_sal = 49900 - sal_used

#Reset df_teams
df_teams['Actual Exposure'] = 0

m = 0 #Completed teams so far
while (m < num_teams):
    #pick 6 players per probabilties
    curr_team = []
#     for p in lock_full_names:
#         curr_team.append(p)
    #print(curr_team)
    df_now = df_teams.copy()
    players = df_now['Name + ID'].tolist()
    probs = df_now['Exposure'].tolist()
    for i in range(players_left):
        p = pick_element_per_probs(players,probs)
        curr_team.append(p)
        df_now = df_now[df_now['Name + ID']!=p]
        players = df_now['Name + ID'].tolist()
        probs = df_now['Exposure'].tolist()
    #Now curr_team is list of 6 names.  Check if salary matches
    df_now = df_teams[df_teams['Name + ID'].isin(curr_team)]
    total_sal = df_now.Salary.sum()
    if (total_sal < min_sal or total_sal > max_sal):
        continue
    
#     print('Salary good for team:',curr_team)
#     print(df_now)
    #Put team in order of decreasing salary
    df_now.sort_values(by='Salary',ascending=False,inplace=True)
    #curr_team = df_now['Name + ID'].tolist()
    curr_team = lock_full_names + df_now['Name + ID'].tolist()
    #print(curr_team)
    #Check if team has shown up before in df_lineups
    df3 = (df_lineups == np.array(curr_team))
    old_team = df3.transpose().all().any() #True if team is already in df_lineups
    if old_team:
        continue
        
    #OK, so now we have a new team!  Add it to df_lineups
    df_lineups.loc[m] = curr_team
    #Also update 'Actual Exposure' in df_teams
    new_players = np.where(df_teams['Name + ID'].isin(curr_team),1,0)
    df_teams['Actual Exposure'] = df_teams['Actual Exposure'] + new_players
    
    #Finally incredment m
    m += 1
    #print('Team {}, sal = {}'.format(m,total_sal))
    
df_teams[['Name + ID','Actual Exposure',
         'Exposure','Salary']]

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Name + ID,Actual Exposure,Exposure,Salary
0,Dustin Johnson (14421203),9,32,10000
1,Patrick Cantlay (14421204),16,38,9800
2,Adam Scott (14421205),8,38,9600
3,Xander Schauffele (14421206),6,44,9400
4,Webb Simpson (14421207),19,50,9200
5,Bryson DeChambeau (14421208),12,44,9100
6,Tommy Fleetwood (14421209),10,50,9000
7,Hideki Matsuyama (14421211),14,50,8800
8,Sungjae Im (14421213),17,44,8600
9,Patrick Reed (14421214),13,44,8500


In [30]:
#If you're good with actual exposure, then write it!
df_lineups.to_csv('Lineups/DK_Entries.csv')

In [31]:
df_lineups['All'] = df_lineups.G1 + df_lineups.G2 + df_lineups.G3 + df_lineups.G4 + df_lineups.G5 + df_lineups.G6 
#df_lineups[(df_lineups.All.str.contains('Casey'))]

In [32]:
df_lineups[(df_lineups.All.str.contains('Higgs'))&
          (df_lineups.All.str.contains('Scott'))&
          (df_lineups.All.str.contains('Knox'))]

,G1,G2,G3,G4,G5,G6,All
